In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tensorflow import keras
import os
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.preprocessing import StandardScaler,RobustScaler
sc = StandardScaler()
rb = RobustScaler()

In [ ]:
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv("../input/spaceship-titanic/test.csv")

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
train

In [ ]:
def clean_data(data, k):
    data['CryoSleep'].replace({False: 0, True: 1}, inplace=True)
    data['VIP'].replace({False: 0, True: 1}, inplace=True)
    data['Destination'].replace({'TRAPPIST-1e': 0, '55 Cancri e': 1, 'PSO J318.5-22': 2}, inplace=True)
    data['HomePlanet'].replace({'Earth': 0, 'Europa': 1, 'Mars': 2}, inplace=True)
    
    data['HomePlanet'].fillna(data['HomePlanet'].mode().iloc[0], inplace=True)
    train['Destination'].fillna(train['Destination'].mode().iloc[0], inplace=True)
    data['CryoSleep'].fillna(data['CryoSleep'].mode().iloc[0], inplace=True)
    data['Age'].fillna(data['Age'].mode().iloc[0], inplace=True)
    data['VIP'].fillna(data['VIP'].mode().iloc[0], inplace=True)
    data['RoomService'].fillna(data['RoomService'].mode().iloc[0], inplace=True)
    data['FoodCourt'].fillna(data['FoodCourt'].mode().iloc[0], inplace=True)
    data['ShoppingMall'].fillna(data['ShoppingMall'].mode().iloc[0], inplace=True)
    data['Spa'].fillna(data['Spa'].mode().iloc[0], inplace=True)
    data['VRDeck'].fillna(data['VRDeck'].mode().iloc[0], inplace=True)
    
    data.drop(['PassengerId','Cabin','Name'], axis=1, inplace=True)
    
    if(k == "Train"):
        data['Transported'].replace({False: 0, True: 1}, inplace=True)
        
    return data

In [ ]:
train = clean_data(train, "Train")
test = clean_data(test, "Test")

target = train.pop('Transported')

In [ ]:
train.isna().sum()

In [ ]:
rb.fit(train)
train = rb.transform(train)
test = rb.transform(test)

In [ ]:
train.shape, test.shape

In [ ]:
EPOCH = 100
BATCH_SIZE = 256

In [ ]:
def build_model():
    model = tf.keras.Sequential()

    model.add(layers.Dense(512, activation='relu', input_shape = (10, )))
    
    model.add(layers.Dense(256, activation='relu', kernel_initializer='random_normal'))
    
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(128, activation='relu', kernel_initializer='random_normal'))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(64, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(32, activation='relu', kernel_initializer='random_normal'))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
save_best = tf.keras.callbacks.ModelCheckpoint("Model.h5", verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
his = model.fit(train, target, validation_split=0.2, epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[save_best])

In [ ]:
model = tf.keras.models.load_model('./Model.h5')

In [ ]:
model.evaluate(train,target)

In [ ]:
pred = pd.DataFrame((model.predict(np.asarray(test), verbose=1)).round())

In [ ]:
pred[0].replace({0: False, 1: True}, inplace=True)

In [ ]:
submission_file = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
submission_file['Transported'] = pred
submission_file.to_csv('submission.csv', index=False)

In [ ]:
submission_file